# **Singular Input**

## ***Loading Dataset***

In [1]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

## ***Data Preprocessing***

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    housing.data, housing.target, random_state=60
)

In [3]:
import tensorflow as tf
tf.random.set_seed(60) # Fixing randomness in the model

## ***Modeling***

In [4]:
# Creating layers
normalization_layer = tf.keras.layers.Normalization() # Normalization layer
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu") # Hidden layer 1
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu") # Hidden layer 2
concat_layer = tf.keras.layers.Concatenate() # Merge layer
output_layer = tf.keras.layers.Dense(1) # Output layer

In [6]:
# Building neural networks
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)
model = tf.keras.Model(inputs=[input_], outputs=[output])

In [7]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 8)]                  0         []                            
                                                                                                  
 normalization (Normalizati  (None, 8)                    17        ['input_2[0][0]']             
 on)                                                                                              
                                                                                                  
 dense (Dense)               (None, 30)                   270       ['normalization[1][0]']       
                                                                                                  
 dense_1 (Dense)             (None, 30)                   930       ['dense[1][0]']         

## ***Model Training***

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
normalization_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20
387/387 [==============================] - 2s 4ms/step - loss: 1.0558 - root_mean_squared_error: 1.0275 - val_loss: 0.5581 - val_root_mean_squared_error: 0.7471
Epoch 2/20
387/387 [==============================] - 1s 3ms/step - loss: 0.4894 - root_mean_squared_error: 0.6996 - val_loss: 0.4614 - val_root_mean_squared_error: 0.6793
Epoch 3/20
387/387 [==============================] - 1s 3ms/step - loss: 0.4189 - root_mean_squared_error: 0.6472 - val_loss: 0.4025 - val_root_mean_squared_error: 0.6344
Epoch 4/20
387/387 [==============================] - 1s 3ms/step - loss: 0.3876 - root_mean_squared_error: 0.6226 - val_loss: 0.4315 - val_root_mean_squared_error: 0.6569
Epoch 5/20
387/387 [==============================] - 1s 3ms/step - loss: 0.3765 - root_mean_squared_error: 0.6136 - val_loss: 0.3818 - val_root_mean_squared_error: 0.6179
Epoch 6/20
387/387 [==============================] - 1s 3ms/step - loss: 0.3604 - root_mean_squared_error: 0.6003 - val_loss: 0.4003 - val_

In [9]:
# Model evaluation
mse_test = model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 2ms/step - loss: 0.3215 - root_mean_squared_error: 0.5670


## ***Prediction***

In [10]:
import pandas as pd
X_new = X_test[:3]
pd.DataFrame(X_new)

,0,1,2,3,4,5,6,7
0,2.6765,20.0,5.300836,1.100279,919.0,2.559889,38.01,-121.32
1,4.1389,39.0,5.010204,1.010204,1108.0,3.768707,33.97,-118.09
2,2.5294,37.0,4.879257,1.034056,1385.0,4.287926,37.46,-122.13


In [11]:
y_pred = model.predict(X_new)
pd.DataFrame(y_pred)

1/1 [==============================] - 0s 127ms/step


,0
0,1.175945
1,1.757604
2,1.212498


In [12]:
# Real values
pd.DataFrame(y_test[:3])

,0
0,0.964
1,1.660
2,1.594


# **Functional API for multiple inputs.**

In [13]:
# Input layers
input_wide = tf.keras.layers.Input(shape=[5])
input_deep = tf.keras.layers.Input(shape=[6])

# Normalization layers
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

## ***Modeling***

In [14]:
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

# Hidden layers
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

# Concatenation layer
concat = tf.keras.layers.concatenate([norm_wide, hidden2])

#Output layer
output = tf.keras.layers.Dense(1)(concat)

# Model building
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

## ***Compiling***

In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20, validation_split=0.2)

Epoch 1/20
387/387 [==============================] - 3s 6ms/step - loss: 1.1296 - root_mean_squared_error: 1.0628 - val_loss: 0.5461 - val_root_mean_squared_error: 0.7390
Epoch 2/20
387/387 [==============================] - 1s 3ms/step - loss: 0.4647 - root_mean_squared_error: 0.6817 - val_loss: 0.4358 - val_root_mean_squared_error: 0.6601
Epoch 3/20
387/387 [==============================] - 1s 3ms/step - loss: 0.4108 - root_mean_squared_error: 0.6410 - val_loss: 0.4078 - val_root_mean_squared_error: 0.6386
Epoch 4/20
387/387 [==============================] - 1s 3ms/step - loss: 0.3874 - root_mean_squared_error: 0.6224 - val_loss: 0.3818 - val_root_mean_squared_error: 0.6179
Epoch 5/20
387/387 [==============================] - 1s 3ms/step - loss: 0.3765 - root_mean_squared_error: 0.6136 - val_loss: 0.3828 - val_root_mean_squared_error: 0.6187
Epoch 6/20
387/387 [==============================] - 1s 3ms/step - loss: 0.3629 - root_mean_squared_error: 0.6024 - val_loss: 0.4025 - val_

In [23]:
# Model evaluation
mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)


162/162 [==============================] - 0s 3ms/step - loss: 0.3287 - root_mean_squared_error: 0.5733


In [24]:
# Model prediction
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]
y_pred = model.predict((X_new_wide, X_new_deep))
pd.DataFrame(y_pred)

1/1 [==============================] - 0s 128ms/step


,0
0,0.983235
1,1.758588
2,1.369819
